In [9]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
from datetime import datetime
import backtrader as bt
import backtrader.feeds as btfeeds
import tushare as ts
import pandas as pd
from IPython.display import (display, display_html, display_png, display_svg)
import matplotlib.pyplot as plt
import numpy as np

class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=25)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)

cerebro = bt.Cerebro()
cerebro.addstrategy(SmaCross)
#从tushare 引入数据
pro = ts.pro_api(token='0a9d3308245c51f14e45b0c6d9166ffcc4fbd13d3438222f99f675f7')
df = pro.daily(ts_code='002020.SZ', start_date='2019101', end_date='20210101').iloc[::-1]
df.trade_date=pd.to_datetime(df.trade_date) #由于trade_date是字符串，BackTrader无法识别，需要转一下
data = btfeeds.PandasData(
    dataname=df,
    fromdate=datetime(2019, 1, 1),
    todate=datetime(2021, 1, 1),
    datetime='trade_date',
    open='open',
    high='high',
    low='low',    
    close='close',
    volume='vol',
    openinterest=-1
)
print(df.head())
cerebro.adddata(data)
# 设置投资金额100000.0
cerebro.broker.setcash(10000.0)
cerebro.addsizer(bt.sizers.FixedSize,stake=500)
cerebro.broker.setcommission(commission=0.001)
# 引擎运行前打印期出资金
print('组合期初资金: %.2f' % cerebro.broker.getvalue())
cerebro.run()
# 引擎运行后打期末资金
print('组合期末资金: %.2f' % cerebro.broker.getvalue())

       ts_code trade_date   open   high    low  close  pre_close  change  \
301  002020.SZ 2019-10-10  10.77  10.78  10.63  10.73      10.72    0.01   
300  002020.SZ 2019-10-11  10.74  10.86  10.73  10.74      10.73    0.01   
299  002020.SZ 2019-10-14  10.85  10.91  10.77  10.82      10.74    0.08   
298  002020.SZ 2019-10-15  10.78  10.81  10.60  10.68      10.82   -0.14   
297  002020.SZ 2019-10-16  10.66  10.97  10.66  10.85      10.68    0.17   

     pct_chg        vol      amount  
301   0.0933  126912.30  136069.243  
300   0.0932  156611.33  168991.652  
299   0.7449  253171.63  273938.956  
298  -1.2939  151708.50  162299.693  
297   1.5918  177508.77  192696.490  
组合期初资金: 10000.00
组合期末资金: 10853.93


In [10]:

cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
results = cerebro.run()
strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

import pyfolio as pf
pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions,
    round_trips=True)

C:\Users\HW\anaconda3\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


Start date,2019-10-10
End date,2020-12-31
Total months,14
,Backtest
Annual return,7.1%
Cumulative returns,8.5%
Annual volatility,15.9%
Sharpe ratio,0.51
Calmar ratio,0.72
Stability,0.49
Max drawdown,-9.8%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'